<img src="https://egc2023.sciencesconf.org/data/pages/logo_2.jpg" alt="EGC 2023" width="200px"/>


# Démonstration - Perdido Geoparser - EGC 2023 

Cette démonstration présente la librairie Python [Perdido](https://github.com/ludovicmoncla/perdido) pour le geoparsing et le geocoding de textes en français. 


**Auteurs** : [Ludovic Moncla](https://ludovicmoncla.github.io) (Univ Lyon, INSA Lyon, CNRS, UCBL, LIRIS, UMR 5205, F-69621)
[Mauro Gaio](https://lma-umr5142.univ-pau.fr/fr/organisation/membres/cv_-mgaio-fr.html) (Université de Pau et des Pays de l'Adour, CNRS, LMAP, UMR 5142)

## 1. En bref


Dans cette démonstration, nous allons voir comment :

- Utiliser la librarie [Perdido](https://github.com/ludovicmoncla/perdido) pour le geoparsing et le geocoding :
  - afficher les entités nommées annotées ;
  - cartographier les lieux geocodés ;
  - illustrer la problématique de désambiguïsation des toponymes.
  - enregistrer les résultats dans différents formats (csv, dataframe, ...)

## 2. Configurer l'environnement


* Si vous avez déjà configuré votre environnement, soit avec conda, soit avec pip (voir le fichier [README.md](https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing/-/blob/main/README.md)), vous pouvez ignorer la section suivante et passer directement à la 3.
* Si vous exécutez ce notebook depuis Google Colab, vous devez exécuter la cellule suivante :

In [ ]:
! pip install perdido==0.1.34
! pip install display-xml==0.1.0

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 3. Perdido Geoparser

`Perdido` est une librairie Python pour le geoparsing de texte en français. Le geoparsing se décompose en deux tâches : le **geotagging** et le **geocoding**. 

Le geotagging est similaire à la tâche de **reconnaissance des entités nommées** avec un focus particulier pour le repérage d'information spatiale. En plus des entités nommées, nous nous intéressons en particuliers aux relations entres ces entités telles que les relations spatiales (distances, topologie, orientation, etc.).

Le geocoding (ou résolution de toponymes) a pour rôle d'attribuer aux entités de lieux des coordonnées géographiques.
`Perdido` s'appuie sur une approche hybride principalement construite à base de règles pour la repérage et la classification des entités nommées. La librairie est disponible en 2 versions : une version standard et une version spécialement adaptée pour les articles encyclopédiques.

Dans cette partie nous allons voir comment utiliser `Perdido` pour le geoparsing.

### 3.1 Importer la librairie

In [ ]:
from perdido.geoparser import Geoparser

### 3.2 Executer le geoparser

In [ ]:
text = "J'aimerais vous proposer un rendez-vous, dans un lieu tenu secret à Lyon, "
text += "proche de la place Bellecour, de la place des Célestins, "
text += "au sud de la fontaine des Jacobins et près du pont Bonaparte."

In [ ]:
geoparser = Geoparser()
doc = geoparser(text)

### 3.3 Visualiser les résultats

* Visualiser les attributs des tokens :

In [ ]:
for token in doc:
    print(f'{token.text}\tlemma: {token.lemma}\tpos: {token.pos}')

* Format IOB :

In [ ]:
for token in doc:
    print(token.iob_format())

* Format IOB-TSV :

In [ ]:
for token in doc:
    print(token.tsv_format())

* Afficher la sortie XML-TEI. Voir la référence ci-après pour une description du schéma de balisage :

> Moncla, L., & Gaio, M. (2015). A multi-layer markup language for geospatial semantic annotations. Proceedings of the 9th Workshop on Geographic Information Retrieval, 1–10. Paris, France. [https://doi.org/10.1145/2837689.2837700](https://doi.org/10.1145/2837689.2837700)

In [ ]:
print(doc.tei)

* Utilisation de la librairie [display_xml](https://github.com/mpacer/display_xml) pour affichage du XML avec coloration syntaxique :

In [ ]:
from display_xml import XML
XML(doc.tei, style='lovelace')

* Afficher la sortie GeoJSON :

In [ ]:
doc.geojson

* Afficher la liste des entités nommées :

In [ ]:
for entity in doc.named_entities:
    print(f'entity: {entity.text}\ttag: {entity.tag}')
    if entity.tag == 'place':
        for t in entity.toponym_candidates:
            print(f' latitude: {t.lat}\tlongitude: {t.lng}\tsource {t.source}')

* Afficher la liste des entités nommées étendues :

In [ ]:
for nested_entity in doc.nested_named_entities:
    print(f'entity: {nested_entity.text}\ttag: {nested_entity.tag}')
    if nested_entity.tag == 'place':
        for t in nested_entity.toponym_candidates:
            print(f' latitude: {t.lat}\tlongitude: {t.lng}\tsource {t.source}')


* Affichage graphique des résultats avec la librairie [spacy](https://spacy.io/usage/visualizers) :

In [ ]:
from spacy import displacy

displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True)

In [ ]:
displacy.render(doc.to_spacy_doc(), style="span", jupyter=True)

### 3.4 Exporter les résultats

* Enregistrer les résultats au format XML-TEI :

In [ ]:
doc.to_xml('filename.xml')

* Enregistrer les résultats au format GeoJSON :

In [ ]:
doc.to_geojson('filename.geojson')

* Enregistrer les résultats au format IOB-TSV :

In [ ]:
doc.to_iob('filename.tsv')

* Enregistrer les résultats au format CSV :

In [ ]:
doc.to_csv('filename.csv')

### 3.5 Paramétrage

La librairie est disponible en 2 versions : une version `Standard` et une version `Encyclopedie` spécialement adaptée pour les articles encyclopédiques. L'étape de géocoding est fortement paramétrable, en particulier afin de filtrer les résultats provenants des ressources géographiques (dans le but de limiter les ambiguïtés).

#### 3.5.1 Paramétrage du geotagging

* Paramétrer la version des règles d'annotation utilisée pour la reconnaissance des entités nommées :

    * `Standard` (par défaut): règles développées initialement pour le traitement de descriptions de randonnées
    * `Encyclopedie` : règles adaptées pour le traitement d'article encyclopédique

In [ ]:
text = "ARQUES, (Géog.) petite ville de France, en Normandie, "
text += "au pays de Caux, sur la petite riviere d'Arques. Long. 18. 50. lat. 49. 54."

geoparser = Geoparser(version="Encyclopedie")
doc = geoparser(text)

displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True)
displacy.render(doc.to_spacy_doc(), style="span", jupyter=True)

#### 3.5.2 Paramétrage du geocoding

* Pour les exemples suivants nous allons utiliser un extrait d'une description de randonnée :

In [ ]:
text  =  "Départ du refuge d'Entre le Lac près du lac de la Plagne."
text += " Du refuge Entre le Lac, un sentier remonte les pentes herbeuses et permet de rejoindre le GR5 un peu avant le chalet de la Grassaz (chalet du berger 2335m)."
text += " Toujours en direction du sud , on remonte le vallon en longeant le ruisseau." 
text += " On parvient ainsi à l'extrémité ouest du lac de Grattaleu ; un peu plus haut, on atteint le refuge du col du Palet (2550m)."
text += " On admire la beauté de la vallée et le sommet de Bellecote recouvert de glaciers"

* On utilisera également la trace GPS pour l'afficher sur la carte et pouvoir comparer avec les lieux géocodés :

In [ ]:
import shapely.wkt

# Créer l'objet géométrie à partir d'un string
gpx_geom = shapely.wkt.loads("MULTILINESTRING ((6.839184 45.480323, 6.83987 45.479299, 6.841973 45.478562, 6.843969 45.47857, 6.845235 45.479931, 6.847187 45.479675, 6.847766 45.480112, 6.848368 45.479932, 6.849569 45.480172, 6.850213 45.479029, 6.850685 45.477254, 6.8509 45.476306, 6.851114 45.474425, 6.851758 45.472889, 6.853217 45.471626, 6.855106 45.46982, 6.855534 45.468766, 6.855191 45.468014, 6.85605 45.46684, 6.856092 45.464914, 6.856049 45.463891, 6.855663 45.462356, 6.856093 45.461679, 6.856822 45.46137, 6.857122 45.460038, 6.857166 45.458864, 6.857724 45.458021, 6.858754 45.457765, 6.859698 45.458503, 6.860513 45.458097, 6.861564 45.458676, 6.861575 45.457732, 6.860641 45.456396, 6.864246 45.456245, 6.865748 45.455944, 6.867508 45.455763, 6.869139 45.455884, 6.870942 45.456034, 6.875319 45.454649, 6.879224 45.455432, 6.881391 45.454996, 6.883816 45.455463, 6.886091 45.455613, 6.887936 45.456004, 6.890725 45.457841, 6.890789 45.45608, 6.891379 45.454687, 6.892742 45.454379, 6.894287 45.453084, 6.896261 45.45191, 6.896733 45.452663, 6.898192 45.452602, 6.899759 45.453009, 6.90184 45.452392, 6.902398 45.451308, 6.903771 45.450827, 6.905102 45.450525, 6.906303 45.451308, 6.909179 45.449742, 6.911582 45.448568, 6.913986 45.447755, 6.916839 45.44619, 6.917944 45.444971, 6.920036 45.444083, 6.92214 45.441493, 6.921732 45.440138, 6.91921 45.437713, 6.916174 45.435741, 6.915831 45.433859, 6.915701 45.432067, 6.914092 45.430877, 6.912398 45.429567, 6.912054 45.428845, 6.910767 45.428423, 6.910209 45.4277, 6.910423 45.427038, 6.908664 45.425321, 6.906861 45.423603, 6.905509 45.42261, 6.903985 45.421586, 6.904544 45.420486, 6.904587 45.418784, 6.903986 45.41779, 6.9039 45.416254, 6.904393 45.414838, 6.9042 45.413301, 6.902741 45.411855, 6.90081 45.411524, 6.900123 45.410636, 6.90111 45.409476, 6.901003 45.408707, 6.900349 45.408564, 6.900064 45.409095, 6.899343 45.408773, 6.89742 45.408511, 6.893514 45.404624, 6.890639 45.403811, 6.889116 45.403374, 6.887592 45.402576, 6.886176 45.401822, 6.886262 45.400858, 6.886444 45.400128, 6.887142 45.4, 6.88652 45.399201, 6.885339 45.398297, 6.885704 45.397966, 6.88403 45.397318, 6.88227 45.396851))")

* Paramétrer la ou les ressources utilisées (gazetier) : 

    * `nominatim` (par défaut): [OpenStreetMap](https://www.openstreetmap.org)
    * `ign` : [GeoPortail](https://www.geoportail.gouv.fr)
    * `geonames` : [Geonames](http://www.geonames.org)
    * `whg`: [World Historical Gazetteer](https://whgazetteer.org)
    * `pleiades`: [Pleiades](https://pleiades.stoa.org)

In [ ]:
geoparser = Geoparser(sources=['ign'])
doc = geoparser(text)
doc.get_folium_map()

* L'objet `Perdido` a un attribut `geometry_layer` qui peut prendre la valeur d'une couche de géométrie à afficher sur la carte. Dans notre cas, on ajoute la géométrie de la trace GPS de la randonnées :

In [ ]:
doc.geometry_layer = gpx_geom
doc.get_folium_map()

* Paramétrer le nombre de résultats retournés pour chaque toponyme (par ressource), 1 par défaut :

In [ ]:
geoparser = Geoparser(max_rows=10)
doc = geoparser(text)
doc.geometry_layer = gpx_geom
doc.get_folium_map()

* Filtrer les résultats par pays (code pays) :

In [ ]:
geoparser = Geoparser(max_rows=10, country_code = 'fr')
doc = geoparser(text)
doc.geometry_layer = gpx_geom
doc.get_folium_map()

* Filtrer les résultats selon une zone géographique (bounding box: `east`,`south`,`west`,`north`) :

In [ ]:
geoparser = Geoparser(max_rows=10, bbox = [5.62216508714297, 45.051683489057, 7.18563279407213, 45.9384576816403])
doc = geoparser(text)
doc.geometry_layer = gpx_geom
doc.get_folium_map()

### 3.6 Désambiguïsation

La librairie Perdido est toujours en cours de développement et d'amélioration dans le cadre de différents projet de recherche ([ANR CHOCUAS](), [GEODE]()), à l'heure actuelle une seule méthode de désambiguïsation automatique est disponible. Il s'agit d'une méthode de filtrage par clustering.


#### 3.6.1 Clustering par densité spatiale

Le principe est de regrouper les résultats en utilisant un algorithme de clustering spatial (DBSCAN, *density-based spatial clustering of applications with noise*) et de selectionner le cluster qui contient le plus d'entités distinctes.

> Moncla, L., Renteria-Agualimpia, W., Nogueras-Iso, J., & Gaio, M. (2014). Geocoding for texts with fine-grain toponyms: an experiment on a geoparsed hiking descriptions corpus. Proceedings of the 22nd ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems, 183–192. Dallas, TX, USA: ACM. [https://doi.org/10.1145/2666310.2666386](https://doi.org/10.1145/2666310.2666386)

La librairie Perdido utilise la méthode DBSCAN implémentée dans la librairie [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html). 

Cette stratégie est adaptée pour des textes où les différents lieux cités sont supposés être localisés à proximité les uns des autres (ex: descriptions de randonnées).

In [ ]:
geoparser = Geoparser(sources=['ign'], max_rows=10, bbox = [5.62216508714297, 45.051683489057, 7.18563279407213, 45.9384576816403])
doc = geoparser(text)

doc.cluster_disambiguation()

doc.geometry_layer = gpx_geom
doc.get_folium_map()

## 4. Perdido Geocoder

En plus de la classe Geoparser, la librairie Perdido propose aussi la classe Geocoder. Cette classe permet de geocoder un ou plusieurs toponymes. Les paramètres sont les mêmes que ceux utilisés avec le geoparser pour configuer l'étape de geocoding :
* sources
* max_row
* country_code
* bbox

L'objet retourné est de type Perdido comme pour le Geoparser ce qui permet d'avoir accès aux mêmes attributs et méthodes que précédemment.

### 4.1 Importer la librairie

In [ ]:
from perdido.geocoder import Geocoder

### 4.2 Executer le geocoder

* Instancier le geocoder :

In [ ]:
geocoder = Geocoder()

* Geocoder un nom de lieu :

In [ ]:
doc = geocoder('Lyon')

* Geocoder une liste de noms de lieux :

In [ ]:
doc = geocoder(['Lyon', 'Annecy', 'Chamonix'])

### 4.3 Visualiser les résultats

* Afficher le résultat GeoJSON :

In [ ]:
print(doc.geojson)

* Afficher la liste des toponymes candidats :

In [ ]:
for t in doc.toponyms: 
    print(f'lat: {t.lat}\tlng: {t.lng}\tsource {t.source}\tsourceName {t.source_name}')

* Récupérer les toponymes candidats sous la forme d'un geodataframe :

In [ ]:
doc.to_geodataframe()

* Afficher la carte des résultats

In [ ]:
doc.get_folium_map()

## 5. Les jeux de données


La libraire [Perdido](https://github.com/ludovicmoncla/perdido) embarque deux jeux de données : 
 1. des articles encyclopédiques (volume 7 de l'Encyclopédie de Diderot et d'Alembert (1751-1772)), fournit par l'[ARTFL](https://encyclopedie.uchicago.edu) dans le cadre du projet [GEODE](https://geode-project.github.io) ;
 2. des descriptions de randonnées (chaque description est associée à sa trace GPS. Elles proviennent du site [www.visorando.fr](https://www.visorando.com) et ont été collectées dans le cadre du projet [ANR CHOUCAS](http://choucas.ign.fr).

### 5.1 Articles encyclopédiques

Le jeu de données des articles encyclopédiques est disponible dans la librairie en deux versions, une version "brute" (articles fournis par l'ARTFL) au format dataframe et une version déjà annotée par Perdido (format PerdidoCollection). 

%%Nous allons charger la version brute et voir comment manipuler un dataframe.

#### 5.1.1 Corpus brut

In [ ]:
from perdido.datasets import load_edda_artfl 

dataset_artfl = load_edda_artfl()
data_artfl = dataset_artfl['data']
data_artfl.head()

In [ ]:
data_artfl.loc[data_artfl['head'] == 'FRONTIGNAN'].text.item()

#### 5.1.2 Corpus traité par Perdido

In [ ]:
from perdido.datasets import load_edda_perdido

dataset_perdido = load_edda_perdido()
data_perdido = dataset_perdido['data']
df = data_perdido.to_dataframe()
df.head()

In [ ]:
doc = data_perdido[0]
displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True) 


### 5.2 Descriptions de randonnées (traitées par Perdido)

In [ ]:
from perdido.datasets import load_choucas_perdido

dataset_choucas = load_choucas_perdido()
data_choucas = dataset_choucas['data']

df = data_choucas.to_dataframe()
df.head()

In [ ]:
doc = data_choucas[2]

In [ ]:
doc.text

In [ ]:
displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True) 

In [ ]:
doc.get_folium_map()

## 6. Pour aller plus loin

1. Tutoriel (en français) Geoparsing : [https://github.com/ludovicmoncla/tutoriel-geoparsing](https://github.com/ludovicmoncla/tutoriel-geoparsing)
2. Tutoriel (en français) présenté lors de l'atelier [Librairies Python et Services Web pour la reconnaissance d’entités nommées et la résolution de toponymes](https://anf-tdm-2022.sciencesconf.org/resource/page/id/11) de la formation CNRS [ANF TDM 2022](https://anf-tdm-2022.sciencesconf.org) : [https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing](https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing)
3. Tutoriel (en anglais) utilisé pour le cours [SunoikisisDC](https://sunoikisisdc.github.io) Summer 2022 Course on [Natural Language Processing (NLP) for historical texts](https://github.com/SunoikisisDC/SunoikisisDC-2021-2022/wiki/SunoikisisDC-Summer-2022-Session-9) (Session 9)